In [1]:
import numpy as np
import os,glob,re
import random
from tqdm.auto import tqdm
import cv2
import pyarrow.parquet as pq
import glob
import math
import cupy as cp
from einops import rearrange
import time as time
import sys
from multiprocessing import Pool
import argparse
from itertools import islice
import logging

In [2]:
# logging.basicConfig(filename='a_tautau.log',format='%(asctime)s - %(message)s', level=logging.INFO)

In [3]:
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [int(c) if c.isdigit() else c for c in re.split('([0-9]+)',s)]

def sort_nicely(l):
    """ Sort the given list in the way that humans expitertools
    """
    l.sort(key=alphanum_key)
 

BATCH_SIZE = 256
def generate(args):
    print(args[0])
    print(args[1])
    pf = pq.ParquetFile(args[0])
    #print(args[0], pf.num_row_groups, pf.metadata, pf.schema)
    path = args[1]
    record_batch = pf.iter_batches(batch_size=BATCH_SIZE)
    #batch = next(record_batch)
    #transform(batch, path)
    
    #print("There are ", count_items(record_batch), "batches")
    
    batchNum = 0
    numImages = 1
    for batch in record_batch:
        #print(type(record_batch))
        #print(args[0], pf.num_row_groups, batchNum)
        
        df = batch.to_pandas()
        #print(args[0],df.shape, "X_jet column number is ", df.columns.get_loc('X_jet'))
        xj = df.columns.get_loc('X_jet')
        yy = df.columns.get_loc('y')
        #print(df.columns)
        alpha = 0

        
        
        #p = batch.to_pandas()
        #p = pf.to_pandas()
        im = cp.array(np.array(np.array(df.iloc[:, xj].tolist()).tolist()).tolist())
        meta = np.array(df.iloc[:,yy])

        im[im < 1.e-3] = 0 #Zero_suppression
        im = rearrange(im, 'b c h w -> b h w c')
        for _z in range(13):
            im[:,:,:,_z] = (im[:,:,:,_z] - im[:,:,:,_z].mean())/(im[:,:,:,_z].std())
            im[:,:,:,_z] = cp.clip(im[:,:,:,_z], a_min = 0, a_max = 500*im[:,:,:,_z].std(axis = (1,2))[:,None,None])

        print("im shape",im.shape)
        im = im.get()
        #start writing images for each parquet record or each jet in this case
        for i in range(meta.shape[0]):
            #if i %100==0:
            #    print(i)
            img = im[i,:,:,:]
            for _z in range(13):
                
                if (img[:,:,_z]==0).all() == False:
                    img[:,:,_z] = 255*(img[:,:,_z])/(img[:,:,_z].max())

            img = img.astype(np.uint8)
            img = rearrange(img, 'h w c -> h ( w c )')
            
            impath = path+"_batchNum"+str(batchNum)+"_"+str(str(alpha)+".png")
            #impath = os.path.join(path,str(str(alpha)+".png"))
            cv2.imwrite(impath , img)
            # if(numImages== pf.num_row_groups):
            #     print("Succesfully finished running over file", args[0])
            #     logging.info(f'Succesfully finished running over file {args[0]}')
            #     pf.close()
            alpha = alpha +1
            numImages +=1
            
        #transform(batch, path, batchNum)
        
        batchNum +=1
        

In [4]:
!ls /pscratch/sd/b/bbbam/conver_to_pn/signal/train

IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0000_train.parquet
IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0001_train.parquet
IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0002_train.parquet
IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0003_train.parquet
IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0004_train.parquet


In [5]:
inputDir = '/pscratch/sd/b/bbbam/conver_to_pn/signal/train'
outputDir = '/pscratch/sd/b/bbbam/png_classification/signal/train'

if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

#files = list(map(lambda x : os.path.join(inputDir,x),os.listdir(inputDir)))

inFiles = []   
outputFiles =[]
count = 0

for file in sorted(os.listdir(inputDir)):
    count +=1
    #if count == 2:
    #    break
    f = os.path.join(inputDir,file)
    inFiles.append(f)
    id = alphanum_key(f)[-2]
    #print(alphanum_key(file)[-2]) 
    outputFile =os.path.join(outputDir,f"boostedtauJets_{id}")
    outputFiles.append(outputFile)


#print(inFiles[:10])
args = list(zip(inFiles,outputFiles))

print("----------------------------------------")
print(args)
tic = time.time()
with Pool(10) as p:
    print("**************",args)
    p.map(generate,args)
toc = time.time()
print("It took {} minutes to run over {} files".format((toc-tic)/60,len(inFiles)))

----------------------------------------
[('/pscratch/sd/b/bbbam/conver_to_pn/signal/train/IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0000_train.parquet', '/pscratch/sd/b/bbbam/png_classification/signal/train/boostedtauJets_0'), ('/pscratch/sd/b/bbbam/conver_to_pn/signal/train/IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0001_train.parquet', '/pscratch/sd/b/bbbam/png_classification/signal/train/boostedtauJets_1'), ('/pscratch/sd/b/bbbam/conver_to_pn/signal/train/IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0002_train.parquet', '/pscratch/sd/b/bbbam/png_classification/signal/train/boostedtauJets_2'), ('/pscratch/sd/b/bbbam/conver_to_pn/signal/train/IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0003_train.parquet', '/pscratch/sd/b/bbbam/png_classification/signal/train/boostedtauJets_3'), ('/pscratch/sd/b/bbbam/conver_to_pn/signal/train/IMG_aToTauTau_Hadronic_tauDR0p4_m3p6To14p8_dataset_2_unbaised_v2_0004_tra

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
!nvidia-smi

Thu Nov  9 23:54:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:C3:00.0 Off |                    0 |
| N/A   36C    P0    54W / 250W |   2980MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install opencv-python --user

In [ ]:
#!nvcc --version

In [ ]:
#!pip install cupy-cuda11x --user

In [ ]:
#!pip install einops --user

In [ ]:
#!pip install tqdm --user

In [ ]:
!nvidia-smi

In [ ]:
!pwd

In [ ]:
#!ls -l /pscratch/sd/r/rchudasa/E2E_samples/pngFiles_13channels_correctTrackerLayerHits/DYToTauTau_M-50_13TeV-powheg_pythia8/AODJets/ | wc -l

In [ ]:
!nvidia-smi

In [ ]:
!pwd

In [ ]:
import glob

In [ ]:
len(glob.glob('/pscratch/sd/r/rchudasa/E2E_samples/pngFiles_13channels_correctTrackerLayerHits/QCD_Pt-15to7000_TuneCP5_Flat_13TeV_pythia8/AODJets/*'))

In [ ]:
len(glob.glob('/pscratch/sd/r/rchudasa/E2E_samples/pngFiles_13channels_correctTrackerLayerHits/DYToTauTau_M-50_13TeV-powheg_pythia8/AODJets-checkv2/*'))

In [ ]:
len(glob.glob('/pscratch/sd/r/rchudasa/E2E_samples/pngFiles_13channels_correctTrackerLayerHits/TTToHadronic_TuneCP5_13TeV_powheg-pythia8/AODJets/*'))

In [ ]:
len(glob.glob('/pscratch/sd/r/rchudasa/E2E_samples/pngFiles_13channels_correctTrackerLayerHits/WJetsToLNu_TuneCP5_13TeV_madgraphMLM-pythia8/AODJets/*'))